# Manejo de la ENOE
## Carga y transformación de datos

In [1]:
# Librerías para lectura y transformación de datos
import pandas as pd
import pyreadstat
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Se leen los datos con el etiquedado provisto por INEGI
carpeta = "C:\\Users\\Usuario\\Desktop\\Artículo\\ENOE\\enoen nuevo\\"
# COE1
coe1t422, meta = pyreadstat.read_dta(carpeta + "ENOEN_COE1T422.dta")
# COE2
coe2t422, meta = pyreadstat.read_dta(carpeta + "ENOEN_COE2T422.dta")
# HOG
hogt422, meta = pyreadstat.read_dta(carpeta + "ENOEN_HOGT422.dta")
# SDEM
sdemt422, meta = pyreadstat.read_dta(carpeta + "ENOEN_SDEMT422.dta")
# VIV
vivt422, meta = pyreadstat.read_dta(carpeta + "ENOEN_VIVT422.dta")

In [4]:
# Estas líneas permitían cambiar los datos de las columnas con la información de cada categoría pero presentan errores y no está bien codificado

# Corregir nombres de las categorías
# duplicated_categories = ['Complemento urbano- Rural']

# # Realiza las correcciones
# for category in duplicated_categories:
#     corrected_category = category.replace(' ', '')  # Elimina el espacio adicional
#     vivt422['cd_a'] = vivt422['cd_a'].cat.rename_categories({category: corrected_category})

In [3]:
# Seleccionando identificadores de acuerdo al INEGI
# Son indicadores que se encuentran en todos los archivos anteriores coe1, coe2, etc.
# Se utilizan para hacer match en cada caso y crear una sola tabla a partir de las 5 dadas 
idvivienda = ['cd_a','ent', 'con', "v_sel"]
idhogar = ['cd_a','ent', 'con', "v_sel", "n_hog", "h_mud"]
idpersona = ["cd_a","ent", "con", "v_sel", "n_hog", "h_mud", "n_ren"]

In [4]:
# Primer merge
# Merge de tablas coet1 y coet2
coet422 = pd.merge(coe1t422, coe2t422, on= idpersona)
# "coet422" contiene todas las columnas de ambos dataframes "coe1t422" y "coe2t422" y todas las filas donde 'idpersona' tenía valores coincidentes en ambos dataframes

In [5]:
# Comprobación de columnas
coet422.columns

Index(['r_def', 'cd_a', 'ent', 'con', 'upm_x', 'd_sem_x', 'n_pro_viv_x',
       'v_sel', 'n_hog', 'h_mud',
       ...
       'p9_h7', 'p9_m7', 'p9_8', 'p9_h8', 'p9_m8', 'ur_y', 'tipo_y',
       'mes_cal_y', 'fac_tri_y', 'fac_men_y'],
      dtype='object', length=247)

In [7]:
# Renombramos variables para identificar que ya hemos unido los datafrmaes "coe1t422" y "coe2t422"
coet422 = coet422.rename(columns={'p1': 'p1coe', 'p3': 'p3coe'})

In [6]:
# La operación "merge" de pandas agrega sufijos como "_x" y "_y" a las columnas para evitar conflictos de nombres si ambas tablas que tienen una columna con el mismo nombre
# Tabulado para revisar variables repetidas 
tabla = pd.crosstab(coet422['n_ent_x'], coet422['n_ent_y']) # 'n_ent' es el número de entrevista
print(tabla)

n_ent_y    1.0    2.0    3.0    4.0    5.0
n_ent_x                                   
1.0      65103      0      0      0      0
2.0          0  64334      0      0      0
3.0          0      0  65395      0      0
4.0          0      0      0  64109      0
5.0          0      0      0      0  63929


In [7]:
# Hay más de dos columnas con el mismo nombre que tienen el sufijo "_y"
# Se eliminan las variables que son iguales devolviendo sus valores originales
# Elimina columnas que terminan con "_y"
columnas_eliminar = [col for col in coet422.columns if col.endswith('_y')]
coet422 = coet422.drop(columns = columnas_eliminar)
# Cambia nombres de columnas que terminan con "_x"
coet422 = coet422.rename(columns={col: col.split('_')[0] for col in coet422.columns if col.endswith('_x')})
# Se imprime el tamaño del dataframe (filas, columnas)
print(coet422.shape)

(322870, 235)


In [8]:
# fusionar la información de personas mayores de 12 años del COE con el reporte del cuestionario sociodemográfico
# Se une sdemt422 con coet422
sdemcoet422 = pd.merge(sdemt422, coet422, on=idpersona)
print(sdemcoet422.shape)

(322870, 342)


In [9]:
# Se repite la limpieza de columnas repetidas
# Eliminación columnas que terminan con "_y"
columnas_eliminar = [col for col in sdemcoet422.columns if col.endswith('_y')]
sdemcoet422 = sdemcoet422.drop(columns = columnas_eliminar)
# Cambia nombres de columnas que terminan con "_x"
sdemcoet422 = sdemcoet422.rename(columns={col: col.split('_')[0] for col in sdemcoet422.columns if col.endswith('_x')})  
print(sdemcoet422.shape)

(322870, 336)


In [10]:
# fusionar la información de vivienda (vivt422) y hogar (hogt422). A una vivienda pueden corresponder varios hogares
vivhogt422 = pd.merge(vivt422, hogt422, on=idvivienda)
# Tamaño del dataframe
print(vivhogt422.shape)

(133108, 57)


In [11]:
# Limpieza de columnas repetidas
# Elimina columnas que terminan con "_y"
columnas_eliminar = [col for col in vivhogt422.columns if col.endswith('_y')]
vivhogt422 = vivhogt422.drop(columns = columnas_eliminar)
# Cambia nombres de columnas que terminan con "_x"
vivhogt422 = vivhogt422.rename(columns={col: col.split('_')[0] for col in vivhogt422.columns if col.endswith('_x')})  
print(vivhogt422.shape)

(133108, 39)


In [12]:
# Se unen los utlimos dos dataframes creados para conformar uno solo "completat422"
completat422 = pd.merge(vivhogt422, sdemcoet422, on= idhogar)
print(completat422.shape)

(322870, 369)


In [13]:
# Elimina columnas que terminan con "_y"
columnas_eliminar = [col for col in completat422.columns if col.endswith('_y')]
completat422 = completat422.drop(columns = columnas_eliminar)
# Cambia nombres de columnas que terminan con "_x"
completat422 = completat422.rename(columns={col: col.split('_')[0] for col in completat422.columns if col.endswith('_x')})  
print(completat422.shape)

(322870, 350)


In [14]:
completat422 = completat422.drop(completat422[(completat422['r_def'] != 0) | (completat422['c_res'] == 2)].index)

In [15]:
print(completat422.columns[completat422.columns.duplicated()].tolist())

['est', 'est', 't', 'n', 'fac', 'p4', 'p4']


In [ ]:
# Este código se usaba para convertir los nobres de estas columnas en categorías pero presenta errores, en cuanto lo corrija lo actualizo
# completat422['sex','niv_ins','t_loc_men',
#              'clase1','clase2','clase3',
#              'rama', 'c_ocu11c',
#              'ing7c','ingocup','ing_x_hrs',
#              'fac','est_d_men','upm'] = completat422['sex', 'eda','niv_ins', 't_loc_men',
#                                                      'clase1','clase2','clase3','rama', 
#                                                      'c_ocu11c','ing7c','ingocup','ing_x_hrs',
#                                                      'fac','est_d_men','upm'].astype('category')

In [16]:
# Se revisan las columnas para identificar que se encuentre completo el dataframe
cols = [col for col in completat422.columns]
cols

['loc',
 'mun',
 'est',
 'est',
 'est',
 'ageb',
 't',
 't',
 'cd_a',
 'ent',
 'con',
 'upm',
 'd',
 'n',
 'v_sel',
 'n',
 'per',
 'p1',
 'p2',
 'p3',
 'ur',
 'tipo',
 'mes',
 'fac',
 'fac',
 'n_hog',
 'h_mud',
 'r_pre',
 'p_dia',
 'p_mes',
 'p_anio',
 'r_def',
 'd_dia',
 'd_mes',
 'd_anio',
 'e_obs',
 'p4',
 'p4',
 'inf',
 'r',
 'est_d_tri',
 'est_d_men',
 't_loc_tri',
 't_loc_men',
 'd_sem',
 'n_pro_viv',
 'n_ent',
 'n_ren',
 'c_res',
 'par_c',
 'sex',
 'eda',
 'nac_dia',
 'nac_mes',
 'nac_anio',
 'l_nac_c',
 'cs_p12',
 'cs_p13_1',
 'cs_p13_2',
 'cs_p14_c',
 'cs_p15',
 'cs_p16',
 'cs_p17',
 'n_hij',
 'e_con',
 'cs_p20a_1',
 'cs_p20a_c',
 'cs_p20b_1',
 'cs_p20b_c',
 'cs_p20c_1',
 'cs_ad_mot',
 'cs_p21_des',
 'cs_ad_des',
 'cs_nr_mot',
 'cs_p23_des',
 'cs_nr_ori',
 'zona',
 'salario',
 'fac_tri',
 'fac_men',
 'clase1',
 'clase2',
 'clase3',
 'pos_ocu',
 'seg_soc',
 'rama',
 'c_ocu11c',
 'ing7c',
 'dur9c',
 'emple7c',
 'medica5c',
 'buscar5c',
 'rama_est1',
 'rama_est2',
 'dur_est',
 'a

## Almacenamiento

In [32]:
# Esta línea transforma y guarda el dataframe completat422 en un archivo csv de forma local
completat422.to_csv("completat422.csv", index = False)